# My Playground for Neural Network

In [453]:
from utils import *
from stratified import *
from run import *

In [454]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [455]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [456]:
def g(x): # sigmoid function
    return np.exp(x) / (1 + np.exp(x))

In [457]:
def initialize_weights(ohe_category,layer_parameter, biasterm=True):
    weight_matrix_list = []

    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category  
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm == True else 0
    
    updatedlayerparameterwbias = [len(inputcategory)+b] + list(np.array(layer_parameter)+b) + [len(outputcategory)] # [inputlayer, layerparameters, outputlayer]
    for i in range(len(updatedlayerparameterwbias)-1):
        layernow = updatedlayerparameterwbias[i]
        layernext = updatedlayerparameterwbias[i+1]
        init_weight = np.random.rand(layernext,layernow) * 2 - 1 # initialize the weight with random number between -1 and 1
        weight_matrix_list.append(init_weight)
        
    return weight_matrix_list

In [458]:
# Forward propagation vectorized
def forward_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 15):
    normed_ohe_copy = normed_hotted_data.copy()
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    # random_indices = np.random.choice(435, size=k, replace=False)
    # random_batch = normed_hotted_data[random_indices].T
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    batchnow = 0
    for onebatch in splitted:
        print(batchnow,'batch')
        onebatch = onebatch.T
        input_data = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].mean(axis=1)
        input_data = np.append(1,input_data) if b == 1 else input_data

        current_layer_a = input_data # input layer is the current layer
        current_layer_index = 0
        for theta in weights_list:
            print(current_layer_index,'layer')
            z = np.dot(theta,current_layer_a)
            a = g(z)
            current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index != len(weights_list)) else a
            current_layer_index += 1
        
        batchnow += 1
    # random_batch_input = random_batch[inputindex].mean(axis=1)

    
    a_lists = [] # list of np.array matrix, attribute of each layer
    for i in range(len(weights_list)):
        pass

    return input_data

In [459]:
w = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
indice = [0,1,3]
w = np.array(w)
z = w[indice]
print(z)
input = z.mean(axis=1)
print(input)
input_data = np.append(1,input)
print(input_data)
haihaihai = np.random.rand(5,3) * 2 - 1
print(haihaihai)
nextz = np.dot(haihaihai,input)
print(nextz)

[[ 1  2  3]
 [ 4  5  6]
 [10 11 12]]
[ 2.  5. 11.]
[ 1.  2.  5. 11.]
[[-0.36065125  0.50793159  0.04274046]
 [ 0.6151486  -0.8718554  -0.85376038]
 [ 0.88521846 -0.66044439  0.78018998]
 [-0.14283832  0.77486233  0.69470604]
 [ 0.27607971 -0.88339884 -0.72146218]]
[  2.28850047 -12.52034393   7.05030477  11.23040141 -11.80091875]


In [460]:
layerdataparameter = [4,5,4] # [First layer, Second layer, Third layer], not including the bias and input & output layer
weights = initialize_weights(ohe_house_category,layerdataparameter)
print([len(i) for i in weights])
kk = forward_network(ohe_house_data,ohe_house_category,weights)
print(kk)

[5, 6, 5, 2]
0 batch


ValueError: shapes (6,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)

In [ ]:
k = 5
wok = np.array([[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16]])
whaha = len(wok)
print(whaha)
print(wok.shape[0])
random_indices = np.random.choice(whaha, size=k, replace=False)
random_rows = wok[random_indices]
print(random_rows)

8
8
[[ 9 10]
 [11 12]
 [ 7  8]
 [ 1  2]
 [13 14]]


In [ ]:
layernow = 3
layernext = 2
init_weight = np.random.rand(layernext,layernow) * 2 - 1
a_now = np.random.rand(layernow,1)
print(init_weight)
print(a_now)
c = np.dot(init_weight,a_now)
print(c)

[[ 0.94890886  0.36993194 -0.35456241]
 [-0.69162407  0.27041438  0.23202255]]
[[0.26314019]
 [0.79083739]
 [0.67633022]]
[[0.3024508 ]
 [0.18878358]]
